In [1]:
#Current Problem

#child problems are depenting on each other, so need to solve them in order and asign tasks to the correct child problem and vice versa

In [2]:
#we gone need many types of BrainFunctions

#Tasks
    #tasks have subtasks
    #if a task couldn't be solved, it can be retried later
    #tasks should consist of descriptions results and parameters

#Atomic Tasks are tasks with no subtasks

#Central control
#    decides what to do next.
#       Split Tast
#       Solve Task
#       Retry Task
#       Improve Task Solution

#TaskSplitter Task->List[Task] (simpler tasks)
#TaskSolver Task->Result + Optional List[Task] (new tasks encountered during solving, debugging, etc.)
#Task Duplicate Detector List[Task]->List[Task]
#Task Merger List[Task]->Task
#Task Selector 
#Function tath gathers all the results that can be helpfull to solve a task
#asign solved task to other tasks that could benefit from the results
#ask for most complex task and split it into subtasks


#JSON Stuffer Takes result of Brainfunction and stuffs it into a JSON string
#retries with "doesnt look like correct JSNO jet" until its successfull parsed into object

In [23]:
import json
import re
from collections import deque
from typing import Dict, Iterable, List, Set, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain

from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

import os
from langchain.chat_models import PromptLayerChatOpenAI
from langchain.schema import HumanMessage

class Task:
    def __init__(self, TaskDescription: str = "dscr", Result:str = "not solved", Solved:bool = False):
        self.Description:str = TaskDescription
        self.Result:str = Result
        self.Solved:bool = Solved
        self.ChildTasks:List[Task] = []
        self.ParentTasks:List[Task] = []
        
    def __str__(self)->str:
        if(self.Solved):
            return "Solved["+self.Description+"]>>"+self.Result
        else:
            return "Unsolved["+self.Description+"]>>"+self.Result
        
    def get_all_tasks(self, visited: Set['Task'] = None) -> List['Task']:
        if visited is None:
            visited = set()
        if self in visited:
            return []
        visited.add(self)
        all_tasks = [self]  # Add the current task to the list
        for subtask in self.ChildTasks:
            all_tasks +=subtask.get_all_tasks(visited)  # Recursively get all subtasks of the subtask
        return all_tasks

    def display(self, indent_level: int = 0) -> None:
        indent = '  ' * indent_level  # Modify the number of spaces in the quotes to adjust indentation
        print(indent + str(self))
        for subtask in self.ChildTasks:
            subtask.display(indent_level + 1)
    
    def is_atomic(self)->bool:
        return len(self.ChildTasks) == 0
    
    def readyToSolve(self)->bool:
            return all(map(lambda x: x.Solved, self.ChildTasks)) #all([]) is True
        
    def add_child(self, child: 'Task') -> None:
        if child in self.ChildTasks:
            return
        self.ChildTasks.append(child)
        child.ParentTasks.append(self)

    def add_childs(self, childs: List['Task']) -> None:
        for child in childs:
            self.add_child(child)

class Taskpool(List[Task]):    
    def __str__(self)->str:
        return ",".join(map(str, self.tasks))
    
    def CreateContextSummery(self)->str:
        return f"/n".join(map(lambda x: x.TaskDescription, self.tasks))
    
    def get_task_ready_to_solve(self)->Task:
        for task in self:
            if(task.Solved):
                continue
            if(task.readyToSolve()):
                return task
        raise Exception("No task ready to solve")
    
    def get_nummerated_task_List(self) -> str:
        formatted_tasks = []
        for index, task in enumerate(self):
            formatted_tasks.append(f"{index}:{task.Description}")
        return "\n".join(formatted_tasks)
    

class Agent():
    def __init__(self, Objectiv: str):
        self.Objectiv:Task = Task(Objectiv)
        self.Taskpool = Taskpool()
        self.Taskpool.append(self.Objectiv)
        # self.chat = ChatOpenAI(temperature=0)
        self.chat = PromptLayerChatOpenAI(temperature=0)

    def extract_json(self,text):
        # Find a JSON object or array pattern in the text
        pattern = r'({.*?}|\[.*?\])'
        match = re.search(pattern, text, re.DOTALL)

        # Check if a match is found
        if match:
            json_str = match.group()
            try:
                # Parse the JSON string into a Python object
                json_obj = json.loads(json_str)
                return json_obj
            except json.JSONDecodeError:
                print("Invalid JSON detected.")
                return None
        else:
            print("No JSON found in the text.")
            return None

    def should_be_split(self, task: Task) -> bool:
        #test if task is atomic if no throw exception
        if len(task.ChildTasks) != 0:
            raise ValueError("Task is not atomic")
        
        print("Now in should_be_slpit:" + task.Description)

        
       

        messages = []
        messages.append(SystemMessage(content=
                                      "You are an AI that determines if a task could be split into multiple simpler tasks. Return 'True' if it could be split and 'False' otherwise. Dont be chaty."))
        
        messages.append(HumanMessage(content=
                                     f"Return 'True' if it could be split and 'False' otherwise. Task description: {task.Description}"))
        
        response = self.chat(messages).content

        # Remove unnecessary characters and convert the response to lowercase
        clean_response = response.strip().lower()
        
        print("SplitTestResponse:" + clean_response)

        # Check if the response contains 'true' or 'false'
        if 'true' in clean_response:
            return True
        elif 'false' in clean_response:
            return False
        else:
            raise ValueError("AI response does not contain a valid boolean string: " + response)


    def SplitTask(self, task:Task, verbose: bool = True) -> List[Task]:
        print("Now in SplitTask:" + task.Description)

        messages = []
        messages.append(SystemMessage(content="""ChatGPT, as a task-splitting AI, please analyze the following main task and provide independent, helpful subtasks that contribute to its successful completion. Keep the response concise and focused. Keep in mind that the subtasks should be independent of each other and can be completed in any order and independently. The main Task will be solved as soon as all subtasks are solved. Create a hierarchical view. Dont create Tasks that are not directly helping in solving the Maintask. Start with the tasks that need to be completed first. Don't expand the scope of the task with the subtasks, take the task literally."""))
        messages.append(HumanMessage(content="""ChatGPT, as a task-splitting AI, please analyze the following main task and provide independent, helpful subtasks that contribute to its successful completion. Keep the response concise and focused. Keep in mind that the subtasks should be independent of each other and can be completed in any order and independently. The main Task will be solved as soon as all subtasks are solved. Create a hierarchical view. Dont create Tasks that are not directly helping in solving the Maintask. Start with the tasks that need to be completed first. Don't expand the scope of the task with the subtasks, take the task literally. Maintask to split: """ + task.Description))

        #call OpenAI
        generation = self.chat(messages)

        print("Splitted Tasks befor JSON packing: " +generation.content)
        
        messages.append(generation)

        messages.append(HumanMessage(content="""Don't expand the scope of the task with the subtasks, take the task literally. Keep Scope as minimal as possible."""))
        
        #call OpenAI
        response = self.chat(messages)

        messages.append(response)

        messages.append(HumanMessage(content="""Now transform this List of Tasks to an JSON Arrey of strings like ["hello", "world"]."""))
        
        #call OpenAI
        JSONSTR = self.chat(messages).content
        print("Splitted Tasks as JSON " +JSONSTR)

        JSON = self.extract_json(JSONSTR)
        
        Tasks=[]

        for task in JSON:
            Tasks.append(Task(TaskDescription=task))

        return Tasks
    
    def DummySolveAtomicTask(self, task:Task, verbose: bool = True) -> Task:
        finalMessages=[]
        finalMessages.append(SystemMessage(content="You are a dummy task solver AI to test Systems. You simulate the solution to an Task, and returns belivable Informations and results. Dont be chaty. Concentrate on the results. Its not important if the Information its true, it just has to look like it is."))
        

        # Check if the task has any child tasks that have been solved
        if task.ChildTasks and all(child_task.Solved for child_task in task.ChildTasks):
            # Include the results of the child tasks in the task description
            task_description = f"Maintask: {task.Description}. The results of the child tasks are: {', '.join(child_task.Result for child_task in task.ChildTasks)}"
            task_description+=" Use these results to solve the main task."
            finalMessages.append(HumanMessage(content=task_description))
    

        finalMessages.append(HumanMessage(content=f"Current Task: {task.Description}"))

        # get a chat completion from the formatted messages
        generated = self.chat(finalMessages).content

        task.Result = generated
        task.Solved = True
        return task
    
    
    
    def ToJSON(self, text, class_obj) -> str:
        chat = ChatOpenAI(temperature=0)

        # Create SystemMessagePromptTemplate
        system_message_prompt = SystemMessagePromptTemplate.from_template(
            "You are a helpful assistant that generates a JSON string based on the given text, without adding any extra fields. The class has the following attributes: {attributes}."
        )

        # Create HumanMessagePromptTemplate
        human_message_prompt = HumanMessagePromptTemplate.from_template("{text}")

        ai_message = AIMessagePromptTemplate.from_template("Here is the JSON string:")

        # Combine MessagePromptTemplates into a ChatPromptTemplate
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt, ai_message])

        # Initialize LLMChain
        chain = LLMChain(llm=chat, prompt=chat_prompt)

       # Create an instance of the class to retrieve its attributes
        instance = class_obj()

        # Retrieve class attributes using the provided code
        attrs = []
        for i in inspect.getmembers(instance):
            if not i[0].startswith('_'):
                if not inspect.ismethod(i[1]):
                    attrs.append(i[0])

        # Build attributes string including collection/array hints
        type_hints = class_obj.__annotations__
        attributes_str = ", ".join([f"{attr} (collection)" if attr in type_hints and self.is_collection_or_array(type_hints[attr]) else f"{attr}" for attr in attrs])

        success = False
        for i in range(5):
            response = chain.run(text=text, attributes=attributes_str)
            print(response)
            try:
                # Attempt to create an instance of the class with the generated JSON string
                parsed_json = json.loads(response)
                instance = class_obj(**parsed_json)
                success = True
                break
            except Exception as e:
                # If instance creation fails, update the prompt and try again
                print(f" There was an error creating an instance: {str(e)}")

        if not success:
            raise Exception("Failed to create instance after 5 attempts.")
        
        return response
    
    def FromJSON(self, json_string, class_obj):
        return class_obj(**json.loads(json_string))
    
    def TextToPO(self, text, class_obj):
        jsonStr = self.ToJSON(text, class_obj)
        return self.FromJSON(jsonStr, class_obj)
    
    def run(self):
        
        #hardcoded passes of possible splitting
        for i in range(1):
            currentTasks = Taskpool()
            currentTasks.extend(self.Taskpool) #create copy of Taskpool or else loop will be endless
            for task in currentTasks:
                if task.is_atomic():
                    #if self.should_be_split(task):
                    splitDone=True
                    print("Splitting Task")
                    newTasks = self.SplitTask(task)
                    task.add_childs(newTasks)
                    self.Taskpool.extend(newTasks)
                    self.Objectiv.display()
                    print("Splitting Done")
        
        print(self.Taskpool.get_nummerated_task_List())

        while self.Objectiv.Solved==False:
            currentTask=self.Taskpool.get_task_ready_to_solve()
            self.DummySolveAtomicTask(currentTask)
            self.Objectiv.display()

In [24]:
agent = Agent("Write a story about a fun adventure by the cast of Frends(Sitcome) that takes place in the Frends(Sitcome) universe.")
agent.run()
print(agent.Objectiv.Result)





Splitting Task
Now in SplitTask:Write a story about a fun adventure by the cast of Frends(Sitcome) that takes place in the Frends(Sitcome) universe.
Splitted Tasks befor JSON packing: Subtasks:
1. Brainstorm ideas for the adventure story.
2. Choose the main characters from the Friends cast to feature in the story.
3. Determine the setting and timeline of the story within the Friends universe.
4. Develop a plot that includes a clear beginning, middle, and end.
5. Create a conflict or obstacle that the characters must overcome during the adventure.
6. Write a rough draft of the story, including dialogue and descriptions of the characters and setting.
7. Edit and revise the rough draft to improve the flow and coherence of the story.
8. Proofread the final draft for grammar and spelling errors.
9. Add any necessary details or elements to enhance the story, such as humor or suspense.
10. Finalize the story and prepare it for publication or sharing.
Splitted Tasks as JSON Sure, here's the JS